In [24]:
import pandas as pd
import numpy as np
import os
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
ignore = ['mr','quite','mmm','sort','actually','cause','sir','gonna','lad','shh','ha','uh','ah','ahhh','ooh','ya', 'so', 'got', 'that', 'this', '000', 'em', 'huh', 'aye', 'dum', 'la', 'ssh', 'okay', 'ok', 'gotta', 'hmm', 'aw', 'ow', 'also', 'yes', 'ah', 'said', 'well', 'would', 'yeah', 'two', 'shut','till', 'shall','john', 'mrs', 'ya', 'gotta', 'bit', 'hi', 'outta', 'bye', 'ii', 'aah','um', 'whoa', 'wanna','wow','uh']
for x in ignore:
    STOPWORDS.add(x)

In [ ]:
from sklearn.datasets import fetch_20newsgroups


In [ ]:
corpus = fetch_20newsgroups(subset='test', categories=['alt.atheism', 'comp.graphics', 'rec.motorcycles', 'sci.space', 'talk.politics.guns'])
documents = corpus.data

In [ ]:
corpus.target

In [ ]:
# 1940-disney = 0
# 1950-disney = 1
# 1960-disney = 2
# 1970-disney = 3
# 1980-disney = 4
# 1990-disney = 5
# 2000-disney = 6
# 2010-disney = 7

In [3]:
data = []
target = []

In [15]:
path = r'C:\Users\Attilio\Desktop\progettone\word cloud subtitles\clean_subtitles\\subtitle disney\Disney Animation\2010'

duemila_disney_file = []
for x, y, z in os.walk(path):
    for a in z:
      #  if ".srt" in a:
        duemila_disney_file.append(x+'\\'+a)


for srt_file in duemila_disney_file:
    with open(srt_file,encoding="utf-8", errors = 'ignore') as f:
        srt_file_as_string = f.read()
    data.append(srt_file_as_string)
    target.append(7)
print(len(data))
print(len(target))

95
95


In [10]:
target

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3]

In [11]:
duemila_disney_file

['C:\\Users\\Attilio\\Desktop\\progettone\\word cloud subtitles\\clean_subtitles\\\\subtitle disney\\Disney Animation\\1970\\A tale of two critters 1977.srt',
 'C:\\Users\\Attilio\\Desktop\\progettone\\word cloud subtitles\\clean_subtitles\\\\subtitle disney\\Disney Animation\\1970\\Herbie.Goes.to.Monte.Carlo 1977.srt',
 'C:\\Users\\Attilio\\Desktop\\progettone\\word cloud subtitles\\clean_subtitles\\\\subtitle disney\\Disney Animation\\1970\\Superdad 1973.srt']

In [16]:
# codice alejandro correlazione
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tsr_functions import *
from sklearn.preprocessing import MultiLabelBinarizer

In [17]:
def get_relevant_words(X, y, nwords, featnames, tsr_function=information_gain_positive):
    nD,nF = X.shape
    positives = y.sum()
    negatives = nD - positives

    # computes the 4-cell contingency tables for each feature
    TP = np.asarray((X[y==1]>0).sum(axis=0)).flatten()
    FN = positives - TP
    FP = np.asarray((X[y==0]>0).sum(axis=0)).flatten()
    TN = negatives - FP
    _4cell = [ContTable(tp=TP[i], tn=TN[i], fp=FP[i], fn=FN[i]) for i in range(nF)]

    # applies the tsr_function to the 4-cell counters
    feat_informativeness = np.array(list(map(tsr_function, _4cell)))

    top_relevant_terms = np.argsort(-feat_informativeness)[:nwords]
    feat_names = np.asarray(featnames)[top_relevant_terms]
    feat_informativeness = feat_informativeness[top_relevant_terms]

    return list(zip(feat_names, feat_informativeness))


In [18]:
documents = data
target = np.array(target)

In [37]:
target

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7])

In [32]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, stop_words = STOPWORDS, max_df=.65)
X = tfidf.fit_transform(data)

In [33]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(target.reshape(-1, 1))
nC = Y.shape[1]
print(Y.shape)

(95, 8)


In [36]:
nwords = 50
for i in range(nC):
    # getting the 10 most relevant terms for each class according to the (positive only) correlation to the
    # group label, as quantified by information gain
    class_i_relevant = get_relevant_words(X, Y[:,i], nwords, tfidf.get_feature_names())
    print(f'class {i}: ' + ', '.join([f'{word} ({rel:.3f})' for word, rel in class_i_relevant]))

class 0: gay (0.155), 300 (0.141), drunk (0.130), spots (0.097), flows (0.096), thumping (0.084), twitter (0.084), clover (0.084), mightiest (0.084), supercolossal (0.084), russian (0.084), spaces (0.084), persisted (0.084), phooey (0.084), provoked (0.084), unprotected (0.084), birdies (0.084), aviation (0.084), promptly (0.084), bombers (0.084), nestled (0.084), housefly (0.084), forbidding (0.084), tremendous (0.080), planted (0.080), struggle (0.080), cure (0.080), gypsy (0.080), deer (0.080), facing (0.069), shattered (0.069), menace (0.069), compare (0.069), quaint (0.069), destruction (0.069), merry (0.068), feet (0.065), mellow (0.056), existed (0.056), colossal (0.056), subconscious (0.056), formidable (0.056), acquainted (0.056), earthquake (0.056), whirl (0.056), spite (0.056), conquer (0.056), suffered (0.056), blasting (0.056), counselor (0.056)
class 1: immediately (0.122), dee (0.101), gracious (0.099), wings (0.098), nightingale (0.096), pardons (0.096), grandchildren (